**Mount Drive & Start Spark**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("BTS_Model_Training") \
    .config("spark.driver.memory", "12g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.sql.shuffle.partitions", "100") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.memory.fraction", "0.8") \
    .getOrCreate()

print("Spark Version:", spark.version)

Mounted at /content/drive
Spark Version: 4.0.2


**Configure Paths**

In [ ]:
import os
import time

FEATURES_PATH = "/content/drive/MyDrive/Machine Learning and Big Data/features_parquet"
MODELS_PATH   = "/content/drive/MyDrive/Machine Learning and Big Data/models"
TABLEAU_PATH  = "/content/drive/MyDrive/Machine Learning and Big Data/tableau_exports"
CHECKPOINT    = "/content/drive/MyDrive/Machine Learning and Big Data/checkpoints"
LOGS_PATH     = "/content/drive/MyDrive/Machine Learning and Big Data/logs"

for path in [MODELS_PATH, TABLEAU_PATH, CHECKPOINT, LOGS_PATH]:
    os.makedirs(path, exist_ok=True)

spark.sparkContext.setCheckpointDir(CHECKPOINT)
print("Paths configured")

Paths configured


**Load Feature Splits**

In [ ]:
from pyspark.sql import functions as F
from pyspark import StorageLevel

train_df = spark.read.parquet(FEATURES_PATH + "/train")
val_df   = spark.read.parquet(FEATURES_PATH + "/val")
test_df  = spark.read.parquet(FEATURES_PATH + "/test")

# Persist train + val — reused across all model fits and evaluations
# test_df not persisted — only used once at final evaluation
train_df.persist(StorageLevel.MEMORY_AND_DISK)
val_df.persist(StorageLevel.MEMORY_AND_DISK)

train_count = train_df.count()
val_count   = val_df.count()
test_count  = test_df.count()

print("Splits loaded")
print("  Train:", train_count)
print("  Val  :", val_count)
print("  Test :", test_count)

Splits loaded
  Train: 2202967
  Val  : 599226
  Test : 601272


**Evaluation Helper**

In [ ]:
from pyspark.ml.evaluation import (
    BinaryClassificationEvaluator,
    MulticlassClassificationEvaluator
)

def evaluate_on_validation(predictions, model_name, duration_sec=None):
    """Evaluate predictions on validation set and return metrics dict."""

    binary_eval = BinaryClassificationEvaluator(
        labelCol="DelayLabel",
        rawPredictionCol="rawPrediction"
    )
    multi_eval = MulticlassClassificationEvaluator(
        labelCol="DelayLabel",
        predictionCol="prediction"
    )

    auc       = round(binary_eval.evaluate(predictions,
                    {binary_eval.metricName: "areaUnderROC"}), 4)
    accuracy  = round(multi_eval.evaluate(predictions,
                    {multi_eval.metricName: "accuracy"}), 4)
    f1        = round(multi_eval.evaluate(predictions,
                    {multi_eval.metricName: "f1"}), 4)
    precision = round(multi_eval.evaluate(predictions,
                    {multi_eval.metricName: "weightedPrecision"}), 4)
    recall    = round(multi_eval.evaluate(predictions,
                    {multi_eval.metricName: "weightedRecall"}), 4)

    print(f"\n  {model_name}")
    print(f"    AUC      : {auc}")
    print(f"    Accuracy : {accuracy}")
    print(f"    F1       : {f1}")
    print(f"    Precision: {precision}")
    print(f"    Recall   : {recall}")
    if duration_sec:
        print(f"    Time (s) : {duration_sec}")

    return {
        "model": model_name, "auc": auc, "accuracy": accuracy,
        "f1": f1, "precision": precision, "recall": recall,
        "train_time_sec": duration_sec
    }

**Baseline 1: Logistic Regression**

In [ ]:
from pyspark.ml.classification import LogisticRegression

print("Training Baseline 1: Logistic Regression")


lr = LogisticRegression(
    featuresCol="scaledFeatures",
    labelCol="DelayLabel",
    maxIter=10,
    regParam=0.01,
    elasticNetParam=0.0
)

t0       = time.time()
lr_model = lr.fit(train_df)
lr_time  = round(time.time() - t0, 2)

lr_preds   = lr_model.transform(val_df)
lr_metrics = evaluate_on_validation(lr_preds, "Logistic Regression", lr_time)

Training Baseline 1: Logistic Regression

  Logistic Regression
    AUC      : 0.999
    Accuracy : 0.9505
    F1       : 0.9487
    Precision: 0.9536
    Recall   : 0.9505
    Time (s) : 42.3


**Baseline 2: Decision Tree**

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

print("Training Baseline 2: Decision Tree")


dt = DecisionTreeClassifier(
    featuresCol="scaledFeatures",
    labelCol="DelayLabel",
    maxDepth=8,
    seed=42
)

t0       = time.time()
dt_model = dt.fit(train_df)
dt_time  = round(time.time() - t0, 2)

dt_preds   = dt_model.transform(val_df)
dt_metrics = evaluate_on_validation(dt_preds, "Decision Tree", dt_time)

Training Baseline 2: Decision Tree

  Decision Tree
    AUC      : 1.0
    Accuracy : 0.9999
    F1       : 0.9999
    Precision: 0.9999
    Recall   : 0.9999
    Time (s) : 50.25


**Baseline 3: Random Forest**

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

print("Training Baseline 3: Random Forest")


rf = RandomForestClassifier(
    featuresCol="scaledFeatures",
    labelCol="DelayLabel",
    numTrees=20,
    maxDepth=8,
    seed=42
)

t0       = time.time()
rf_model = rf.fit(train_df)
rf_time  = round(time.time() - t0, 2)

rf_preds   = rf_model.transform(val_df)
rf_metrics = evaluate_on_validation(rf_preds, "Random Forest", rf_time)

Training Baseline 3: Random Forest

  Random Forest
    AUC      : 0.9999
    Accuracy : 0.9969
    F1       : 0.9969
    Precision: 0.9969
    Recall   : 0.9969
    Time (s) : 115.74


**Baseline 4: Naive Bayes**

In [ ]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.feature import MinMaxScaler

print("Training Baseline 4: Naive Bayes")


mm_scaler       = MinMaxScaler(inputCol="features", outputCol="nnFeatures")
mm_scaler_model = mm_scaler.fit(train_df)
train_nn        = mm_scaler_model.transform(train_df)
val_nn          = mm_scaler_model.transform(val_df)

nb = NaiveBayes(
    featuresCol="nnFeatures",
    labelCol="DelayLabel",
    smoothing=1.0
)

t0       = time.time()
nb_model = nb.fit(train_nn)
nb_time  = round(time.time() - t0, 2)

nb_preds   = nb_model.transform(val_nn)
nb_metrics = evaluate_on_validation(nb_preds, "Naive Bayes", nb_time)

Training Baseline 4: Naive Bayes

  Naive Bayes
    AUC      : 0.4049
    Accuracy : 0.7433
    F1       : 0.6409
    Precision: 0.8097
    Recall   : 0.7433
    Time (s) : 21.6


**Baseline Comparison & Select Best**

In [ ]:
all_metrics = [lr_metrics, dt_metrics, rf_metrics, nb_metrics]

print("\nBaseline Comparison — Validation Set")
print(f"{'Model':<25} {'AUC':>6} {'Accuracy':>10} {'F1':>8} {'Time(s)':>9}")
print("-" * 62)
for m in all_metrics:
    print(f"  {m['model']:<23} {m['auc']:>6} {m['accuracy']:>10} "
          f"{m['f1']:>8} {m['train_time_sec']:>9}")



Baseline Comparison — Validation Set
Model                        AUC   Accuracy       F1   Time(s)
--------------------------------------------------------------
  Logistic Regression      0.999     0.9505   0.9487      42.3
  Decision Tree              1.0     0.9999   0.9999     50.25
  Random Forest           0.9999     0.9969   0.9969    115.74
  Naive Bayes             0.4049     0.7433   0.6409      21.6


**Hyperparameter Tuning with CrossValidator**

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder


evaluator = BinaryClassificationEvaluator(
    labelCol="DelayLabel",
    metricName="areaUnderROC"
)

tuning_est = DecisionTreeClassifier(
    featuresCol="scaledFeatures",
    labelCol="DelayLabel",
    seed=42
)

param_grid = ParamGridBuilder() \
    .addGrid(tuning_est.maxDepth,            [5]) \
    .addGrid(tuning_est.minInstancesPerNode, [1, 5]) \
    .build()

cv = CrossValidator(
    estimator=tuning_est,
    estimatorParamMaps=param_grid,
    evaluator=evaluator,
    numFolds=3,
    parallelism=4,
    seed=42
)

# Checkpoint before CV — truncates DAG lineage for iterative fitting
train_df.checkpoint()

t0          = time.time()
cv_model    = cv.fit(train_df)
tuning_time = round(time.time() - t0, 2)
best_tuned  = cv_model.bestModel

print(f"\nTuning complete in {tuning_time}s")
print(f"CV AUC per param set : {[round(s, 4) for s in cv_model.avgMetrics]}")
print(f"Best CV AUC          : {round(max(cv_model.avgMetrics), 4)}")


Tuning complete in 421.56s
CV AUC per param set : [np.float64(0.9994), np.float64(0.9994)]
Best CV AUC          : 0.9994


In [ ]:
tuned_val_preds   = best_tuned.transform(val_df)
tuned_val_metrics = evaluate_on_validation(
    tuned_val_preds, "Decision Tree Tuned (Val)", tuning_time
)


  Decision Tree Tuned (Val)
    AUC      : 0.9996
    Accuracy : 0.9951
    F1       : 0.9951
    Precision: 0.9951
    Recall   : 0.9951
    Time (s) : 421.56


In [ ]:
importances = best_tuned.featureImportances.toArray()

numerical_cols = [
    "Month", "DayofMonth", "DayOfWeek", "Quarter",
    "DepHour", "ArrHour", "CRSElapsedTime", "Distance",
    "DistanceGroup", "DepDelay", "DepDelayMinutes",
    "TaxiOut", "TaxiIn", "AirTime",
    "CarrierDelay", "WeatherDelay", "NASDelay",
    "SecurityDelay", "LateAircraftDelay",
    "RouteAvgDelay", "AirlineDelayRate",
    "IsWeekend", "IsHolidayMonth", "DelayRiskScore"
]

n  = min(len(numerical_cols), len(importances))
fi = sorted(zip(numerical_cols[:n], importances[:n]), key=lambda x: -x[1])

print("Top 15 Feature Importances (Tuned Model — Validation):")
print(f"{'Feature':<30} {'Importance':>10}")
print("-" * 42)
for feat, imp in fi[:15]:
    print(f"  {feat:<28} {round(imp, 4):>10}")

feature_importance_data = [(f, round(float(i), 4)) for f, i in fi]

Top 15 Feature Importances (Tuned Model — Validation):
Feature                        Importance
------------------------------------------
  DepDelay                         0.6023
  NASDelay                         0.3079
  CarrierDelay                     0.0599
  LateAircraftDelay                0.0291
  WeatherDelay                     0.0008
  Month                               0.0
  DayofMonth                          0.0
  DayOfWeek                           0.0
  Quarter                             0.0
  DepHour                             0.0
  ArrHour                             0.0
  CRSElapsedTime                      0.0
  Distance                            0.0
  DistanceGroup                       0.0
  DepDelayMinutes                     0.0


In [ ]:

import numpy as np
from sklearn.linear_model import LogisticRegression      as SK_LR
from sklearn.tree          import DecisionTreeClassifier as SK_DT
from sklearn.ensemble      import RandomForestClassifier as SK_RF
from sklearn.naive_bayes   import GaussianNB             as SK_NB
import pickle

SAMPLE_FRACTION = 0.10
SAMPLE_SEED     = 42

# ── Collect sample to driver ─────────────────────────
print(f"Collecting {SAMPLE_FRACTION*100:.0f}% sample to driver...\n")

def collect_features_labels(df, fraction, seed):
    """Sample a Spark DataFrame and return numpy X, y arrays."""
    sample = df.sample(fraction=fraction, seed=seed) \
               .select("scaledFeatures", "DelayLabel")
    rows   = sample.collect()
    X = np.array([r["scaledFeatures"].toArray() for r in rows])
    y = np.array([int(r["DelayLabel"])           for r in rows])
    return X, y

X_train, y_train = collect_features_labels(train_df, SAMPLE_FRACTION, SAMPLE_SEED)
X_val,   y_val   = collect_features_labels(val_df,   SAMPLE_FRACTION, SAMPLE_SEED)

print(f"  Train sample : {X_train.shape[0]:,} rows | {X_train.shape[1]} features")
print(f"  Val   sample : {X_val.shape[0]:,} rows   | {X_val.shape[1]} features")

# ── Define sklearn models ────────────────────────────
sk_models = [
    ("Logistic Regression", SK_LR(
        max_iter=100, C=100,
        solver="saga", n_jobs=-1,
        random_state=42)),
    ("Decision Tree",       SK_DT(
        max_depth=8,
        random_state=42)),
    ("Random Forest",       SK_RF(
        n_estimators=20, max_depth=8,
        n_jobs=-1, random_state=42)),
    ("Naive Bayes",         SK_NB()),
]

# ── Fit each model & record training time ────────────
print("Training sklearn models on single node...\n")

sk_train_results = []
sk_fitted_models = {}

for name, model in sk_models:
    t0      = time.time()
    model.fit(X_train, y_train)
    sk_time = round(time.time() - t0, 2)

    sk_fitted_models[name] = model

    sk_train_results.append({
        "model":          name,
        "framework":      "sklearn_single_node",
        "train_time_sec": float(sk_time),
        "sample_rows":    int(X_train.shape[0])
    })

    print(f"  {name:<25} trained in {sk_time}s")

# ── Save fitted sklearn models with pickle ───────────
SK_MODELS_PATH = os.path.join(MODELS_PATH, "sklearn")
os.makedirs(SK_MODELS_PATH, exist_ok=True)

for name, model in sk_fitted_models.items():
    filename = name.lower().replace(" ", "_") + ".pkl"
    with open(os.path.join(SK_MODELS_PATH, filename), "wb") as f:
        pickle.dump(model, f)

print(f"\nSklearn models saved → {SK_MODELS_PATH}")
print("  logistic_regression.pkl")
print("  decision_tree.pkl")
print("  random_forest.pkl")
print("  naive_bayes.pkl")

# ── Save val sample arrays for NB4 evaluation ────────
# NB4 will load these instead of re-collecting from Spark
np.save(os.path.join(SK_MODELS_PATH, "X_val.npy"),   X_val)
np.save(os.path.join(SK_MODELS_PATH, "y_val.npy"),   y_val)
np.save(os.path.join(SK_MODELS_PATH, "X_train.npy"), X_train)
np.save(os.path.join(SK_MODELS_PATH, "y_train.npy"), y_train)

print("\nVal/Train arrays saved:")
print(f"  X_val.npy   : {X_val.shape}")
print(f"  y_val.npy   : {y_val.shape}")
print(f"  X_train.npy : {X_train.shape}")
print(f"  y_train.npy : {y_train.shape}")

# ── Add training results to NB3 log ──────────────────
# Stored in variable — appended to log dict in Cell 16
sklearn_train_results = sk_train_results

print("\nSklearn training complete — evaluation in Notebook 4")


  Train sample : 220,543 rows | 49 features
  Val   sample : 59,882 rows   | 49 features
Training sklearn models on single node...



/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


  Logistic Regression       trained in 41.7s
  Decision Tree             trained in 2.66s
  Random Forest             trained in 6.36s
  Naive Bayes               trained in 0.47s

Sklearn models saved → /content/drive/MyDrive/Machine Learning and Big Data/models/sklearn
  logistic_regression.pkl
  decision_tree.pkl
  random_forest.pkl
  naive_bayes.pkl

Val/Train arrays saved:
  X_val.npy   : (59882, 49)
  y_val.npy   : (59882,)
  X_train.npy : (220543, 49)
  y_train.npy : (220543,)

Sklearn training complete — evaluation in Notebook 4


**Save Models (MLlib Serialisation)**

In [ ]:
lr_model.write().overwrite().save(MODELS_PATH + "/lr_baseline")
dt_model.write().overwrite().save(MODELS_PATH + "/dt_baseline")
rf_model.write().overwrite().save(MODELS_PATH + "/rf_baseline")
nb_model.write().overwrite().save(MODELS_PATH + "/nb_baseline")
best_tuned.write().overwrite().save(MODELS_PATH + "/best_tuned_dt")

print("All models saved (MLlib format):")
print("  lr_baseline | dt_baseline | rf_baseline")
print("  nb_baseline | best_tuned_dt")
print("  Location:", MODELS_PATH)

All models saved (MLlib format):
  lr_baseline | dt_baseline | rf_baseline
  nb_baseline | best_tuned_dt
  Location: /content/drive/MyDrive/Machine Learning and Big Data/models


**Tableau CSVs: Dashboard 2 (Training Metrics)**

In [ ]:
# Baseline comparison on validation
spark.createDataFrame(
    [(str(m["model"]), float(m["auc"]), float(m["accuracy"]),
      float(m["f1"]), float(m["precision"]), float(m["recall"]), float(m["train_time_sec"]))
     for m in all_metrics],
    ["model", "auc", "accuracy", "f1", "precision", "recall", "train_time_sec"]
).coalesce(1).write.mode("overwrite") \
 .option("header", "true") \
 .csv(TABLEAU_PATH + "/d2_baseline_model_comparison")

# Baseline vs tuned on validation
spark.createDataFrame([
    ("Decision Tree Baseline",
     float(dt_metrics["auc"]), float(dt_metrics["accuracy"]), float(dt_metrics["f1"])),
    ("Decision Tree Tuned (Val)",
     float(tuned_val_metrics["auc"]), float(tuned_val_metrics["accuracy"]), float(tuned_val_metrics["f1"])),
], ["model", "auc", "accuracy", "f1"]
).coalesce(1).write.mode("overwrite") \
 .option("header", "true") \
 .csv(TABLEAU_PATH + "/d2_tuned_vs_baseline_val")

# CV fold scores — avgMetrics returns numpy float64, must cast explicitly
spark.createDataFrame(
    [(int(i + 1), float(s)) for i, s in enumerate(cv_model.avgMetrics)],
    ["param_set", "avg_auc"]
).coalesce(1).write.mode("overwrite") \
 .option("header", "true") \
 .csv(TABLEAU_PATH + "/d2_cv_scores")

# Feature importance
spark.createDataFrame(
    [(str(f), float(i)) for f, i in feature_importance_data],
    ["feature", "importance"]
).coalesce(1).write.mode("overwrite") \
 .option("header", "true") \
 .csv(TABLEAU_PATH + "/d2_feature_importance")

print("D2 Tableau CSVs saved:")
print("  d2_baseline_model_comparison")
print("  d2_tuned_vs_baseline_val")
print("  d2_cv_scores")
print("  d2_feature_importance")

D2 Tableau CSVs saved:
  d2_baseline_model_comparison
  d2_tuned_vs_baseline_val
  d2_cv_scores
  d2_feature_importance
